In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [4]:
lista_ligas = [
    "https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats",
    "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats",
    "https://fbref.com/en/comps/11/2021-2022/2021-2022-Serie-A-Stats",
    "https://fbref.com/en/comps/13/2021-2022/2021-2022-Ligue-1-Stats",
    "https://fbref.com/en/comps/20/2021-2022/2021-2022-Bundesliga-Stats",
]

years = list(range(2021, 2013, -1))
all_matches = []
log_error = []
log = []
for year in years:
    for indice, liga in enumerate(lista_ligas):
        data = requests.get(liga)
        soup = BeautifulSoup(data.text)

        try:
            standings_table = soup.select("table.stats_table")[0]
        except Exception as e:
            
            continue

        links = [l.get("href") for l in standings_table.find_all("a")]
        links = [l for l in links if "/squads/" in l]
        team_urls = [f"https://fbref.com{l}" for l in links]

        previous_season = soup.select("a.prev")[0].get("href")
        

        time.sleep(1)
        for team_url in team_urls:
            
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            data = requests.get(team_url)
            time.sleep(1)
            try:
                matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
                matches = matches[
                    [
                        "Date",
                        "Time",
                        "Comp",
                        "Round",
                        "Venue",
                        "Result",
                        "GF",
                        "GA",
                        "Opponent",
                        "Poss",
                        "Formation",
                    ]
                ]
                soup = BeautifulSoup(data.text)
                links = [l.get("href") for l in soup.find_all("a")]
                time.sleep(1)
            except Exception as e:
                log_error.append(f"error:{e},{lista_ligas[indice]},{team_url}")
                continue

            try:
                links_shooting = [l for l in links if l and "all_comps/shooting/" in l]
                data_shooting = requests.get(f"https://fbref.com{links_shooting[0]}")
                shooting = pd.read_html(data_shooting.text, match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()
                time.sleep(1)
            except Exception as e:
                
                continue

            try:
                links_misc = [l for l in links if l and "all_comps/misc/" in l]
                data_misc = requests.get(f"https://fbref.com{links_misc[0]}")
                defensive = pd.read_html(data_misc.text, match="Miscellaneous Stats")[0]
                defensive.columns = defensive.columns.droplevel()
                time.sleep(1)

            except Exception as e:
                continue

            try:
                team_data = matches.merge(defensive[["Date", "TklW", "Int"]], on="Date")
            except ValueError:
                continue

            try: 
                if 'xG' in shooting.columns:
                    team_data = team_data.merge(
                    shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt", "xG"]],
                        on="Date",
                    )
                    print('entre en el if')
                    print(shooting.columns)
                else:
                    shooting["xG"] = ''
                    team_data = team_data.merge(
                    shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt", "xG"]], on="Date"
                )
                    print('entre en el else')
                    print(shooting.columns)

            except Exception as e:
              
                continue

            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
            time.sleep(2)
            pass

        lista_ligas[indice] = f"https://fbref.com{previous_season}"
     

match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df = match_df[['date', 'time', 'season', 'team', 'comp', 'round', 'venue', 'result', 'gf', 'ga',
       'opponent', 'poss', 'formation', 'tklw', 'int', 'sh', 'sot', 'dist',
       'pk', 'pkatt', 'xg']]
match_df.to_csv('matches.csv')